In [52]:
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time

In [53]:
# this version is more transferable, as compared to the next cell (which relies on path on my computer)
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [/Users/annasours/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [31]:
# URLs of pages to be scraped

# NASA Mars News
url_1 = "https://mars.nasa.gov/news/"
# JPL Mars Space Images - Featured Image
url_2 = "https://www.jpl.nasa.gov/images?search=&category=Mars"
# Mars Facts
url_3 = "https://space-facts.com/mars/"
# Mars Hemispheres
url_4 = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

### Nasa Mars News

In [4]:
browser.visit(url_1)

# one way to get the webpage to load slower
time.sleep(2)

In [6]:
html=browser.html
news_soup=BeautifulSoup(html,'html.parser')

first_article=news_soup.find('ul', class_='item_list')
# get_text
# text.strip
news_title=first_article.find('div', class_='content_title').a.text
news_p=first_article.find('div', class_='article_teaser_body').text

print(news_title)
print('-----'*5)
print(news_p)

Another First: Perseverance Captures the Sounds of Driving on Mars
-------------------------
NASA’s newest rover recorded audio of itself crunching over the surface of the Red Planet, adding a whole new dimension to Mars exploration.


### JPL Mars Space Image - Featured Image

In [54]:
browser.visit(url_2)

# one way to get the webpage to load slower
time.sleep(1)

##### Pull first image 

In [8]:
# browser.find_by_css('img.BaseImage').first.click()
# browser.links.find_by_partial_text('JPG').click()
# featured_img_url=browser.url
# # print(feat_img_url)

##### Alternative: Pull ref image from the dropdown menu

In [55]:
# this only pulls the reference image from the drop down menu. Instead, you can just pull the firs tsource image

dropdown_content=browser.find_by_css('.-active')

for button in dropdown_content:
    button.click()

browser.links.find_by_partial_text('Featured Image').click()
browser.links.find_by_partial_text('JPG').click()
featured_image_url=browser.url
print(featured_image_url)

https://d2pn8kiwq2w21t.cloudfront.net/original_images/jpegPIA24179.jpg


### Mars Facts

In [56]:
import pandas as pd

In [57]:
tables = pd.read_html(url_3)
mars_facts_df=tables[0]
mars_facts_df.columns=['Metric', 'measurement']
mars_facts_df

,Metric,measurement
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [58]:
html_table = mars_facts_df.to_html(index=False)
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Metric</th>\n      <th>measurement</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

In [59]:
# if more needed:
# strip unwanted newlines to clean it up
# html_table.replace('\n', '') 

### Mars Hemispheres

In [60]:
browser.visit(url_4)
time.sleep(1)

In [16]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')


photos = soup.find_all('div', class_='description')

base_url='https://astrogeology.usgs.gov'
img_dict_list=[]


for each_photo in photos:
#     find img_url
    h3 = each_photo.find('h3')
    link = each_photo.find('a')
    href = link['href']
    title=h3.text

    print(f'Getting {title} link')

#     this will click on each photo link.
    browser.links.find_by_partial_text(title).click()
    pic_html = browser.html
    pic_soup = BeautifulSoup(pic_html, 'html.parser')
    source_photo=pic_soup.find('img', class_="wide-image")['src']
    img_url=(base_url+source_photo)

#     create dictionary with title and img url and append to list of dictionaries
    img_dict_list.append({'title': title, 'img_url': img_url})
    
#     go back to original page to follow next link
# option: browser.back()
    browser.visit(url_4)
print('Hemispheres complete')

Getting Cerberus Hemisphere Enhanced link
Getting Schiaparelli Hemisphere Enhanced link
Getting Syrtis Major Hemisphere Enhanced link
Getting Valles Marineris Hemisphere Enhanced link
Hemispheres complete


In [17]:
print(img_dict_list)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]


#### Alternative

In [23]:
# hemi_image_urls = []

# # Next, loop through those links, click the link, find the sample anchor, return the href
# for i in range(4):
#     hemi = {}
    
#     # We have to find the elements on each loop to avoid a stale element exception
#     browser.find_by_css("a.product-item h3")[i].click()
    
#     html = browser.html
#     hemi_soup = BeautifulSoup(html, "html.parser")
    
#     # Next, we find the Sample image anchor tag and extract the href
#     hemi['img_url'] = hemi_soup.find("a", text="Sample").get("href")
    
#     # Get Hemisphere title
#     hemi['title'] = hemi_soup.find("h2", class_="title").get_text()
    
#     # Append hemisphere object to list
#     hemi_image_urls.append(hemi)
    
#     # Finally, we navigate backwards
#     browser.back()

#### Alternative

In [26]:
# hemisphere_image_urls = []

# # Next, loop through those links, click the link, find the sample anchor, return the href
# links = browser.find_by_css("a.product-item h3")

# for i in range(len(links)):
#     hemisphere = {}
    
#     browser.find_by_css("a.product-item h3")[i].click()
    
#     # Next, we find the Sample image anchor tag and extract the href
#     sample_elem = browser.links.find_by_text('Sample').first
#     hemisphere['img_url'] = sample_elem['href']
    
#     # Get Hemisphere title
#     hemisphere['title'] = browser.find_by_css("h2.title").text
    
#     # Append hemisphere object to list
#     hemisphere_image_urls.append(hemisphere)
    
#     # Finally, we navigate backwards
#     browser.back()